# 🏠 DeepRoof-2026: Master Training Lab

### 🛠 Step 1: Super-Nuclear Environment Initialization
This cell handles venv activation, mmseg patching, and **MMCV binary verification** (resolving '_ext' errors).

In [ ]:
import os
import sys
import subprocess
import torch
import ctypes
import glob
from pathlib import Path

# --- 1. PROJECT PATHS ---
project_root = Path("/workspace/roof")
if not project_root.exists():
    project_root = Path(os.getcwd()).parent

venv_path = project_root / "venv"
if not venv_path.exists():
    venv_path = project_root / ".venv"

if venv_path.exists():
    lib_dir = list(venv_path.glob("lib/python*/site-packages"))
    if lib_dir:
        if str(lib_dir[0]) not in sys.path:
            sys.path.insert(0, str(lib_dir[0]))
        sys.executable = str(venv_path / "bin" / "python")
    print(f"🐍 Venv Activated: {venv_path}")
else:
    print("⚠️ No venv found. Using system environment.")

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# --- 2. PRE-EMPTYIVE LIB LOADING (LD_LIBRARY_PATH FIX) ---
def ensure_cuda_libs():
    cuda_available = torch.cuda.is_available()
    if not cuda_available: return
    
    # Search for libcudart in common places
    search_roots = [
        "/usr/local/cuda/lib64/libcudart.so*",
        str(venv_path / "lib/python*/site-packages/nvidia/cuda_runtime/lib/libcudart.so*")
    ]
    
    found_lib = None
    for sr in search_roots:
        matches = glob.glob(sr)
        if matches:
            found_lib = matches[0]
            break
            
    if found_lib:
        print(f"📍 Linking CUDA Runtime: {found_lib}")
        try:
            ctypes.CDLL(found_lib, mode=ctypes.RTLD_GLOBAL)
            lib_dir = os.path.dirname(found_lib)
            os.environ['LD_LIBRARY_PATH'] = lib_dir + os.pathsep + os.environ.get('LD_LIBRARY_PATH', '')
        except: pass
    else:
        print("📦 Installing nvidia-cuda-runtime-cu11...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "nvidia-cuda-runtime-cu11"])
        return False
    return True

# --- 3. MMCV & MMSEG HAMMER PATCH ---
def apply_patches():
    print("🩹 Verifying MMCV & MMSeg integrity...")
    
    # A. mmsegmentation assertion removal
    target_file = None
    try:
        import mmseg
        target_file = Path(mmseg.__file__).parent / "__init__.py"
    except:
        # Fallback search if import fails due to assertions
        matches = glob.glob(str(venv_path / "lib/python*/site-packages/mmseg/__init__.py"))
        if matches: target_file = Path(matches[0])
    
    if target_file and target_file.exists():
        with open(target_file, 'r') as f: content = f.read()
        if "assert (mmcv_min_version" in content:
            print(f"🔥 Deleting assertions in {target_file}")
            unlocked = """# Copyright (c) OpenMMLab. All rights reserved.
import mmcv
import mmengine
from mmengine.utils import digit_version
from .version import __version__, version_info
MMCV_MIN = '2.0.0rc4'
MMCV_MAX = '2.2.0'
MMENGINE_MIN = '0.7.1'
MMENGINE_MAX = '1.0.0'
mmcv_min_version = digit_version(MMCV_MIN)
mmcv_max_version = digit_version('9.9.9') # OVERRIDE by DeepRoof
mmcv_version = digit_version(mmcv.__version__)
mmengine_min_version = digit_version(MMENGINE_MIN)
mmengine_max_version = digit_version('9.9.9') # OVERRIDE by DeepRoof
mmengine_version = digit_version(mmengine.__version__)
__all__ = ['__version__', 'version_info', 'digit_version']
"""
            with open(target_file, 'w') as f: f.write(unlocked)
            return False # Restart needed

    # B. MMCV Extension Check
    if torch.cuda.is_available():
        try:
            import mmcv
            from mmcv.ops import point_sample
            print("✅ MMCV Extensions (Ops) verified.")
        except (ImportError, ModuleNotFoundError) as e:
            print(f"❌ MMCV Ops Error: {e}")
            print("🔄 Reinstalling MMCV Binary for CUDA 11.8 / Torch 2.1...")
            # Force uninstall broken version
            subprocess.run([sys.executable, "-m", "pip", "uninstall", "-y", "mmcv"], check=False)
            # Install known-good binary for this container's architecture
            subprocess.check_call([sys.executable, "-m", "pip", "install", "mmcv==2.2.0", "-f", "https://download.openmmlab.com/mmcv/dist/cu118/torch2.1/index.html"])
            return False

    return True

if ensure_cuda_libs() and apply_patches():
    print(f"🚀 System Ready | Device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")
else:
    print("⚠️ DISK UPDATED. PLEASE RESTART KERNEL NOW.")

## 📂 1. Dataset Preview

Visualize the **OmniCity** satellite imagery and ground truth **Masks** + **Surface Normals**.

In [ ]:
def preview_dataset(num_samples=3):
    import matplotlib.pyplot as plt
    import numpy as np
    import cv2
    
    data_path = project_root / "data/OmniCity"
    train_file = data_path / 'train.txt'
    
    if not train_file.exists():
        print(f"❌ Multi-task training data not found at {data_path}.")
        return
        
    with open(train_file, 'r') as f:
        sample_ids = [line.strip() for line in f.readlines()[:num_samples]]
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5 * num_samples))
    for i, sid in enumerate(sample_ids):
        img = cv2.cvtColor(cv2.imread(str(data_path / 'images' / (sid + '.jpg'))), cv2.COLOR_BGR2RGB)
        
        mask = cv2.imread(str(data_path / 'masks' / (sid + '.png')), cv2.IMREAD_UNCHANGED)
        mask_vis = cv2.applyColorMap(((mask % 20) * 12).astype(np.uint8), cv2.COLORMAP_JET)
        
        axes[i, 0].imshow(img); axes[i, 0].set_title(sid); axes[i, 0].axis('off')
        axes[i, 1].imshow(mask_vis); axes[i, 1].set_title("Mask"); axes[i, 1].axis('off')
        
        norm_path = data_path / 'normals' / (sid + '.npy')
        if norm_path.exists():
            normals = np.load(str(norm_path))
            axes[i, 2].imshow(((normals + 1) * 127.5).astype(np.uint8))
        axes[i, 2].set_title("Normals"); axes[i, 2].axis('off')
        
    plt.tight_layout(); plt.show()

preview_dataset(num_samples=2)

## ⚙️ 2. Scratch Training Configuration (Epoch-Based)

We are using the **MASTER EPOCH-BASED SCRATCH PROFILE**:
- **Duration**: 150 Epochs (~160k steps).
- **Val Interval**: Every 1 Epoch (Reports results per-epoch).
- **No Pre-Training**: `load_from = None`.
- **Checkpoints**: Interval snapshots every 5 epochs + `best_mIoU.pth`.

In [ ]:
from mmengine.config import Config

CONFIG_PATH = str(project_root / "configs/deeproof_scratch_swin_L.py")
WORK_DIR = str(project_root / "work_dirs/swin_l_scratch_v1")

cfg = Config.fromfile(CONFIG_PATH)
cfg.work_dir = WORK_DIR
cfg.data_root = str(project_root / "data/OmniCity/")
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_root = cfg.data_root

print(f"🏆 MASTER SCRATCH CONFIG LOADED")
print(f"🔥 Max Epochs: {cfg.train_cfg.max_epochs}")
print(f"📉 Initial LR: {cfg.optimizer.lr}")
print(f"📊 Reporting Interval: Every Epoch")

## 🚀 3. Kickoff Training

This will invoke the `mmengine.Runner` and begin the full model convergence process. **Detailed stats will print to this output at the end of every epoch.**

In [ ]:
import torch
from mmengine.runner import Runner

print(f"🚀 Starting Master Trainer on: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

runner = Runner.from_cfg(cfg)
runner.train()

## 📊 4. Monitoring & Metrics

Run this cell during or after training to visualize performance trends.

In [ ]:
import json
import matplotlib.pyplot as plt

def plot_training_logs(log_path):
    if not os.path.exists(log_path):
        print("🕒 No logs found yet.")
        return
        
    iters, losses, miou = [], [], []
    with open(log_path, 'r') as f:
        for line in f:
            data = json.loads(line)
            if 'loss' in data:
                iters.append(data.get('iter', 0))
                losses.append(data['loss'])
            if 'mIoU' in data:
                miou.append(data['mIoU'])
                
    plt.figure(figsize=(10, 5))
    plt.plot(iters, losses, label='Loss')
    plt.title("Training Progress"); plt.show()

# log_json = glob.glob(os.path.join(WORK_DIR, "*/vis_data/scalars.json"))
# if log_json: plot_training_logs(log_json[-1])